In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from config import db_password
from flask import Flask
from flask_sqlalchemy import sqlalchemy
import os
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
path = 'Resources/'
office = pd.read_csv(path +'offices_clean.csv', index_col=0)
office

,object_id,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
0,c:1,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253
1,c:3,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945
2,c:4,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
3,c:5,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801
4,c:7,SF Bay,Suite 200,654 High Street,Palo Alto,94301,CA,ISR,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
112713,f:15098,New York,8 Sound Shore Drive,Suite 303,Greenwich,06830,CT,USA,0.000000,0.000000
112714,c:286200,Santa Barbara,735 State Street,Suite 500,Santa Barbara,93101,CA,USA,0.000000,0.000000
112715,c:256895,Los Angeles,"5777 W. Century Blvd., Suite 360",NaN,Los Angeles,90045,CA,USA,0.000000,0.000000
112716,c:256200,New York,420 LExington Avenue,NaN,New York,10170,NY,USA,0.000000,0.000000


In [3]:
office['country_code'].value_counts()

USA    55602
GBR     7960
IND     4065
CAN     4004
DEU     2154
FRA     1876
AUS     1544
ESP     1190
ISR     1189
IRL     1111
NLD      949
CHN      817
SWE      783
ITA      683
FIN      618
CHE      607
SGP      587
BRA      581
JPN      539
RUS      494
DNK      418
BEL      412
TUR      362
ARG      330
HKG      316
POL      309
ZAF      299
NOR      270
AUT      266
NZL      230
KOR      216
PRT      216
UKR      201
ARE      198
ROM      194
MYS      159
MEX      155
TWN      153
PAK      151
CZE      139
IDN      128
GRC      126
PHL      121
EST      113
BGR      104
CHL      103
THA       97
HUN       90
NGA       84
EGY       79
LUX       77
LTU       74
VNM       73
COL       65
CYP       60
ISL       59
KEN       57
LVA       57
HRV       56
CSS       53
BGD       53
JOR       45
SVN       42
URY       39
SAU       38
LKA       37
GHA       33
BLR       32
SVK       32
LBN       32
NPL       27
CRI       27
MLT       26
PER       24
CYM       24
ANT       21
KWT       20

In [4]:
25675-15336

10339

In [5]:
office['city'].value_counts()

New York               4401
0                      4360
San Francisco          3858
London                 3432
Chicago                1151
                       ... 
Owesnboro                 1
Doncaster, Victoria       1
Glasgow,                  1
St. Georgen               1
Guragaon                  1
Name: city, Length: 11075, dtype: int64

In [6]:
objects = pd.read_csv(path +'objects_clean.csv', index_col=0)
objects

C:\Users\phsstudent\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,WA,Seattle,Seattle,NaN,NaN,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,CA,Culver City,Los Angeles,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,CA,San Mateo,SF Bay,NaN,NaN,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462647,r:9996,Product,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462648,r:9997,Product,c:14164,"Cmail, broadcast email marketing",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462649,r:9998,Product,c:14164,"CatylistCRM, contact database",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


In [7]:
usa_only = objects[objects['country_code']=='USA']
usa_only

,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,WA,Seattle,Seattle,NaN,NaN,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,CA,Culver City,Los Angeles,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,CA,San Mateo,SF Bay,NaN,NaN,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
8,c:10005,Company,NaN,Thomas Publishing,advertising,operating,NaN,NaN,USA,NY,New York,New York,NaN,NaN,0,0,NaN,NaN,0,0.0,1999-06-01,1999-06-01,1,2
12,c:10009,Company,NaN,dimension5 labs,advertising,operating,2008-08-01,NaN,USA,NM,Santa Fe,Santa Fe,NaN,NaN,0,0,NaN,NaN,0,0.0,2008-08-22,2008-08-22,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208191,f:9985,FinancialOrg,NaN,Bee Partners,NaN,operating,2008-04-01,NaN,USA,CA,San Francisco,SF Bay,2007-10-15,2013-09-04,12,12,NaN,NaN,0,0.0,2013-05-01,2013-05-01,1,1
208196,f:9991,FinancialOrg,NaN,Recruit Strategic Partners,NaN,operating,1960-01-01,NaN,USA,CA,Palo Alto,SF Bay,2012-06-01,2013-06-18,5,5,NaN,NaN,0,0.0,NaN,NaN,0,0
208202,f:9997,FinancialOrg,NaN,Axel Johnson,NaN,operating,1920-01-01,NaN,USA,NY,New York,New York,2012-10-24,2012-10-24,1,1,NaN,NaN,0,0.0,NaN,NaN,0,0
208203,f:9998,FinancialOrg,NaN,Liberty City Ventures,NaN,operating,2012-07-01,NaN,USA,NY,New York,New York,2012-06-01,2013-11-11,5,5,NaN,NaN,0,0.0,NaN,NaN,0,0


In [8]:
usa_office = office[office['object_id'].isin(usa_only['object_id'])]
usa_office

,object_id,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
0,c:1,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253
1,c:3,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945
2,c:4,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
3,c:5,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801
5,c:8,SF Bay,"959 Skyway Road, Suite 200",NaN,San Carlos,94070,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...
112713,f:15098,New York,8 Sound Shore Drive,Suite 303,Greenwich,06830,CT,USA,0.000000,0.000000
112714,c:286200,Santa Barbara,735 State Street,Suite 500,Santa Barbara,93101,CA,USA,0.000000,0.000000
112715,c:256895,Los Angeles,"5777 W. Century Blvd., Suite 360",NaN,Los Angeles,90045,CA,USA,0.000000,0.000000
112716,c:256200,New York,420 LExington Avenue,NaN,New York,10170,NY,USA,0.000000,0.000000


In [9]:
usa_office2 = usa_office[usa_office['object_id'].str.contains('c:', regex=True)]
usa_office2

,object_id,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
0,c:1,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253
1,c:3,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945
2,c:4,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
3,c:5,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801
5,c:8,SF Bay,"959 Skyway Road, Suite 200",NaN,San Carlos,94070,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...
112712,c:286186,Sacramento,"930 Riverside Pkwy., Ste. 10",NaN,West Sacramento,95605,CA,USA,0.000000,0.000000
112714,c:286200,Santa Barbara,735 State Street,Suite 500,Santa Barbara,93101,CA,USA,0.000000,0.000000
112715,c:256895,Los Angeles,"5777 W. Century Blvd., Suite 360",NaN,Los Angeles,90045,CA,USA,0.000000,0.000000
112716,c:256200,New York,420 LExington Avenue,NaN,New York,10170,NY,USA,0.000000,0.000000


In [10]:
usa_office2[usa_office2['object_id']=='c:9977']

,object_id,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
6108,c:9977,SF Bay,2 Townsend Street,NaN,San Francisco,94107,CA,USA,37.782101,-122.389519


In [11]:
pdr = pd.read_csv(path +'new_degrees.csv', index_col=0)
pdr

,person_object_id,relationship_object_id,title,first_name,last_name,affiliation_name,degree_type,subject,institution,graduated_at
0,p:7,c:4,Chief Executive Officer,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992
1,p:7,c:82,Chairman & Co - Founder,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992
2,p:7,c:23139,CEO,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992
3,p:7,c:38317,Board of Directors,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992
4,p:7,c:68967,Advisor,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992
...,...,...,...,...,...,...,...,...,...,...
16682,p:268498,c:202221,Founder & CEO,Hector,M Morell,"InvierteMe,SL",0.0,Licenciado en Ciencias del Trabajo,Universidad de Sevilla,NaN
16683,p:268502,c:202221,Director of Operations,Víctor,Martínez Calvo,"InvierteMe,SL",0.0,Industrial,Escuela Superior de Ingenieros de Sevilla,NaN
16684,p:268510,c:39172,Executive Chairman,Robert,"L. ""Chip"" Harris",Unaffiliated,2.0,NaN,University of Southern California,1/1/1980
16685,p:268510,c:39172,President,Robert,"L. ""Chip"" Harris",Unaffiliated,2.0,NaN,University of Southern California,1/1/1980


In [15]:
usa = usa_office2.rename(columns={'object_id':'relationship_object_id'})
usa

,relationship_object_id,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
0,c:1,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253
1,c:3,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945
2,c:4,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
3,c:5,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801
5,c:8,SF Bay,"959 Skyway Road, Suite 200",NaN,San Carlos,94070,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...
112712,c:286186,Sacramento,"930 Riverside Pkwy., Ste. 10",NaN,West Sacramento,95605,CA,USA,0.000000,0.000000
112714,c:286200,Santa Barbara,735 State Street,Suite 500,Santa Barbara,93101,CA,USA,0.000000,0.000000
112715,c:256895,Los Angeles,"5777 W. Century Blvd., Suite 360",NaN,Los Angeles,90045,CA,USA,0.000000,0.000000
112716,c:256200,New York,420 LExington Avenue,NaN,New York,10170,NY,USA,0.000000,0.000000


In [18]:
pdrc = pdr.merge(usa, on='relationship_object_id', how='inner')
pdrc

,person_object_id,relationship_object_id,title,first_name,last_name,affiliation_name,degree_type,subject,institution,graduated_at,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude
0,p:7,c:4,Chief Executive Officer,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
1,p:9,c:4,Systems Engineering Manager,Ron,Gorodetzky,Digg,2.0,Computer Engineering,"University of California, San Diego (UCSD)",1/1/2003,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
2,p:6,c:4,Founder & CEO,Kevin,Rose,i/o Ventures,0.0,NaN,"University of Nevada, Las Vegas",NaN,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
3,p:64863,c:4,Senior Software Engineer,Kurt,Wilms,Digg,2.0,Computer Science,University of Minnesota,1/1/2005,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
4,p:110860,c:4,Sr. Developer,Ryan,Marshall,Unaffiliated,0.0,NaN,University of Illinois at Chicago,NaN,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,p:268332,c:64809,President and Chief Executive Officer,Wes,Bush,Unaffiliated,0.0,Executive Management Program,University of California,NaN,Detroit,12001 Tech Center Drive,NaN,Livonia,48150,MI,USA,0.000000,0.000000
6803,p:268332,c:64809,Systems Engineer,Wes,Bush,Unaffiliated,0.0,Executive Management Program,University of California,NaN,Detroit,12001 Tech Center Drive,NaN,Livonia,48150,MI,USA,0.000000,0.000000
6804,p:268334,c:6786,Team Member,Elliott,Counts,Unaffiliated,0.0,NaN,The University of Texas at Austin,NaN,SF Bay,Soma,NaN,San Francisco,94103,CA,USA,37.775392,-122.412214
6805,p:268510,c:39172,Executive Chairman,Robert,"L. ""Chip"" Harris",Unaffiliated,2.0,NaN,University of Southern California,1/1/1980,Los Angeles,500 Newport Center Drive,7th Floor,Newport Beach,92660,CA,USA,0.000000,0.000000


In [20]:
location = pd.read_csv(path +'locations_complete2.csv', index_col=0)
location

,object_id,name,address1,address2,city,zip_code,state_code,country_code,Latitude,Longitude,region
0,c:10005,Thomas Publishing,Five Penn Plaza,15th Floor,New York,10001,NY,USA,40.750422,-73.996328,NY
1,c:222678,ProjectVision,1521 Shattuck Ave,NaN,SF,94611,CA,USA,37.879455,-122.268853,NorCal
2,c:100155,MTPV,13915 Burnet Road,Suite 420,Austin,78728,TX,USA,30.440575,-97.696996,Southwest
3,c:60694,Salter Labs,100 W. Sycamore Road,NaN,LA,93203,CA,USA,35.195170,-118.824708,SoCal
4,c:278710,Avenal Community Health Center,1000 Skyline Blvd.,NaN,LA,93204,CA,USA,36.007680,-120.121415,SoCal
...,...,...,...,...,...,...,...,...,...,...,...
37545,c:215695,Cloudvu,9100 So 500 W,NaN,Sandy,84070,UT,USA,40.586277,-111.905607,West
37546,c:215698,Rotech Healthcare,2600 Technology Drive,Suite 300,Orlando,32804,FL,USA,28.555189,-81.412294,South
37547,c:215699,Nephron Pharmaceuticals,4121 SW 34th St,Orlando,Florida,32811,FL,USA,28.505936,-81.429768,South
37548,c:215749,Bioreclamation Holdings,123 FROST STREET,SUITE 115,WESTBURY,11590,NY,USA,40.759948,-73.554506,NY


In [23]:
loc_prep = location.rename(columns={'object_id':'relationship_object_id'})
loc_prep.head()

,relationship_object_id,name,address1,address2,city,zip_code,state_code,country_code,Latitude,Longitude,region
0,c:10005,Thomas Publishing,Five Penn Plaza,15th Floor,New York,10001,NY,USA,40.750422,-73.996328,NY
1,c:222678,ProjectVision,1521 Shattuck Ave,NaN,SF,94611,CA,USA,37.879455,-122.268853,NorCal
2,c:100155,MTPV,13915 Burnet Road,Suite 420,Austin,78728,TX,USA,30.440575,-97.696996,Southwest
3,c:60694,Salter Labs,100 W. Sycamore Road,NaN,LA,93203,CA,USA,35.195170,-118.824708,SoCal
4,c:278710,Avenal Community Health Center,1000 Skyline Blvd.,NaN,LA,93204,CA,USA,36.007680,-120.121415,SoCal


In [24]:
pdrl = pdr.merge(loc_prep, on='relationship_object_id', how='inner')
pdrl

,person_object_id,relationship_object_id,title,first_name,last_name,affiliation_name,degree_type,subject,institution,graduated_at,name,address1,address2,city,zip_code,state_code,country_code,Latitude,Longitude,region
0,p:7,c:23139,CEO,Jay,Adelson,Digg,2.0,"Film and Broadcasting, Computer Science Concen...",Boston University,1/1/1992,SimpleGeo,41 Decatur,NaN,SF,94103,CA,USA,37.772386,-122.405756,NorCal
1,p:2693,c:23139,Board Observer,Satish,Dharmaraj,Zimbra,0.0,NaN,Harvard Business School,NaN,SimpleGeo,41 Decatur,NaN,SF,94103,CA,USA,37.772386,-122.405756,NorCal
2,p:7608,c:23139,Lead Architect,Mike,Malone,Pownce,2.0,NaN,Virginia Tech,1/1/2002,SimpleGeo,41 Decatur,NaN,SF,94103,CA,USA,37.772386,-122.405756,NorCal
3,p:55160,c:23139,Geographer,Seth,Fitzsimmons,Flickr,2.0,Computer Science,Amherst College,1/1/2002,SimpleGeo,41 Decatur,NaN,SF,94103,CA,USA,37.772386,-122.405756,NorCal
4,p:203866,c:23139,Lead Architect,Michael,Malone,Unaffiliated,2.0,NaN,Virginia Polytechnic and State University,1/1/2006,SimpleGeo,41 Decatur,NaN,SF,94103,CA,USA,37.772386,-122.405756,NorCal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,p:268332,c:64809,President and Chief Executive Officer,Wes,Bush,Unaffiliated,0.0,Executive Management Program,University of California,NaN,TRW,12001 Tech Center Drive,NaN,Livonia,48150,MI,USA,42.372260,-83.340351,Midwest
3576,p:268332,c:64809,Systems Engineer,Wes,Bush,Unaffiliated,0.0,Executive Management Program,University of California,NaN,TRW,12001 Tech Center Drive,NaN,Livonia,48150,MI,USA,42.372260,-83.340351,Midwest
3577,p:268396,f:15091,co-founder,Sammy,Abdullah,Unaffiliated,2.0,Finance,The University of Texas at Austin - The Red Mc...,1/1/2005,Dallas Angel Network,NaN,5307 E. Mockingbird Lan,Dallas,0,TX,USA,32.781179,-96.790329,Southwest
3578,p:268510,c:39172,Executive Chairman,Robert,"L. ""Chip"" Harris",Unaffiliated,2.0,NaN,University of Southern California,1/1/1980,Acacia Research,500 Newport Center Drive,7th Floor,LA,92660,CA,USA,33.615831,-117.871206,SoCal
